In [2]:
using LightGraphs, Distributions, DataFrames, ProgressMeter, RCall

In [3]:
srand(20130810)

MersenneTwister(UInt32[0x01332bfa], Base.dSFMT.DSFMT_state(Int32[-1772545288, 1073534108, 1077066014, 1072915095, -2146195133, 1072843413, 301764553, 1073404181, 750472136, 1073628106  …  -1491411563, 1073194977, 716119449, 1072893711, 1632331784, 758890923, 1433693833, -13012230, 382, 0]), [1.13855, 1.44875, 1.62854, 1.95593, 1.42871, 1.69517, 1.37496, 1.20532, 1.96306, 1.73616  …  1.04, 1.53714, 1.04249, 1.66099, 1.53771, 1.4206, 1.36477, 1.10325, 1.39659, 1.01425], 382)

*Note: This workbook replicates the results from "The chilling effects of network externalities" by Goldenberg, Libai and Muller (2007). This is a self-didactic attempt*

> Two factors influence the transition of a non-adopter to an adopter:
- External factors: Some probability $a$ exists such that in a given time period, an individual will be influenced by external influence mechanisms such as advertising, mass media, and other marketing efforts, to adopt the innovative product
- Internal factors: Some probability $b$ exists such that during a given time period, an individual will be affected by an interaction (e.g., word of mouth) with exactly one other individual who has already adopted the product

> Let the number of cumulative adopters at time $t$ in a market of size $N$ be $x(t)$ and the threshold of an individual $i$ be $h_i$. If $i$ is connected to $m_i(t)$ adopters belonging to her personal network, her probability of adoption is:
$p_i(t) = 1 - (1 - a)(1 - b)^{m_i(t)}$ if $x(t)/N > h_i$ and 0 otherwise.

> In order to compare growth processes with and without network effects, we chose to express our measure as the ratio of the NPV of the growth process with and without network effects. Thus, we compute the NPV for the non-externalities case and for the externalities case using a 10% discount rate per period, which is a reasonable yearly rate for many
markets and fixed profit margins. 

> We assume that the threshold distribution follows a truncated normal distribution with mean $\mu$ and standard deviation $\sigma$.

It follows from the above verbatim from the paper that the parameters are $(a, b, \mu, \sigma)$.

In [4]:
mutable struct Network
    G::LightGraphs.SimpleGraphs.SimpleGraph{Int}
    node_status::BitVector
    threshold::Vector{Float64}
end

In [5]:
function network_externalities_effect(N::Network, node::Int, a::Float64, b::Float64)
    if sum(N.node_status)/nv(N.G) > N.threshold[node]
        n_active_nbrs = sum(N.node_status[neighbors(N.G, node)]) 
        return 1 - (1 - a)*((1 - b)^n_active_nbrs)
    else
        return 0
    end
end

network_externalities_effect (generic function with 1 method)

In [47]:
function evolve_with_externalities!(N::Network, a::Float64, b::Float64)
    for node in shuffle(vertices(N.G))
        if rand(Uniform()) < network_externalities_effect(N, node, a, b)
            N.node_status[node] = true
        end
    end
    
    return nothing
end

evolve_with_externalities! (generic function with 1 method)

In [48]:
function evolve_without_externalities!(N::Network, a::Float64)
    for node in shuffle(vertices(N.G))
        if rand(Uniform()) < a
            N.node_status[node] = true
        end
    end
    
    return nothing
end

evolve_without_externalities! (generic function with 1 method)

In [54]:
function simulate(n::Int, z::Int; T = 30, n_realizations = 10)
    parameter_space = [(a, b, mu, sigma) for a in linspace(0.005, 0.05, 5),
                                             b in linspace(0.05, 0.25, 5),
                                             mu in linspace(0.01, 0.2, 5),
                                             sigma in linspace(0.005, 0.025)]
    output = DataFrame(r = Int[], t = Int[], 
                       a = Float64[], b = Float64[], mu = Float64[], sigma = Float64[], 
                       n_engaged_externalities = Int[], n_engaged_no_externalities = Int[])
    
    @showprogress 1 "Simulating..." for (a, b, mu, sigma) in parameter_space
        for r in 1:n_realizations
            g = erdos_renyi(n, z/n)
            N = Network(g, falses(nv(g)), rand(TruncatedNormal(mu, sigma, 0, Inf), nv(g)))
            
            for t in 1:T
                evolve_with_externalities!(N, a, b)
                n_engaged_externalities = sum(N.node_status)
                
                N.node_status = falses(nv(g))
                evolve_without_externalities!(N, a)
                n_engaged_no_externalities = sum(N.node_status)
                
                push!(output, [r, t, a, b, mu, sigma, n_engaged_externalities, n_engaged_no_externalities])
            end
        end
    end
    
    return output
end

simulate (generic function with 1 method)

In [55]:
result = simulate(625, 8)

Simulating...100%|██████████████████████████████████████| Time: 0:04:05


,r,t,a,b,mu,sigma,n_engaged_externalities,n_engaged_no_externalities
1,1,1,0.005,0.05,0.01,0.005,0,3
2,1,2,0.005,0.05,0.01,0.005,4,4
3,1,3,0.005,0.05,0.01,0.005,8,3
4,1,4,0.005,0.05,0.01,0.005,5,1
5,1,5,0.005,0.05,0.01,0.005,1,1
6,1,6,0.005,0.05,0.01,0.005,1,4
7,1,7,0.005,0.05,0.01,0.005,6,4
8,1,8,0.005,0.05,0.01,0.005,4,3
9,1,9,0.005,0.05,0.01,0.005,3,3
10,1,10,0.005,0.05,0.01,0.005,4,2


In [57]:
@rput result

,r,t,a,b,mu,sigma,n_engaged_externalities,n_engaged_no_externalities
1,1,1,0.005,0.05,0.01,0.005,0,3
2,1,2,0.005,0.05,0.01,0.005,4,4
3,1,3,0.005,0.05,0.01,0.005,8,3
4,1,4,0.005,0.05,0.01,0.005,5,1
5,1,5,0.005,0.05,0.01,0.005,1,1
6,1,6,0.005,0.05,0.01,0.005,1,4
7,1,7,0.005,0.05,0.01,0.005,6,4
8,1,8,0.005,0.05,0.01,0.005,4,3
9,1,9,0.005,0.05,0.01,0.005,3,3
10,1,10,0.005,0.05,0.01,0.005,4,2
